- Need to find LARGE sample of datapoints
  - Using MIDI files - http://www.piano-midi.de
- Classical Music (no lyrics - most in public domain - high quality recordings)
- Solo Music only - for simplicity
- Rewriting http://www.hexahedria.com/2015/08/03/composing-music-with-recurrent-neural-networks/ https://github.com/PyDataFortWayne/biaxial-rnn-music-composition
- Generate new works in the style of a certain composer
- Need Amazon instance?
- Use updated NN package (TensorFlow2 or PyTorch)
  - Research possible replacement package
- Train with Saloon music? (Separate and together!)


Next Steps:
- Research possible replacement package (mido for MIDI and ...)
    
- Review source code 
- Start coding!
- Documentation is for winners!
- ...
- Profit!


Composers:
- Mozart
- Mozart + Saloon mashup! (Stretch Goal!)





*Goals:*
Presentation: March 2020
- August last week = Decide on replacement package
- September 1 = Import and Process MIDI files
- Sep 2 = PyData
- Sep 3 = Translate Code from Theano to PyTorch
- Sep 4 = Translate code
- Oct 1 = Translate Code
- Oct 2 = Translate code
- Oct 3 = Train Model (local)
- Oct 4 = PyData
- Nov 1 = Validate Model (local)
- Nov 1 = Validate Model / Tweaks
- Nov 2-3 = AWS Train
- December 1-4 = Writeup / Cleanup Presentation



**Presentation:**
- Music Theory for Dummies (a.k.a. Grant)
- MIDI file structure / format
- How / which model used

In [4]:
import mido # MIDI file processing with mido package: https://pypi.org/project/mido/
import pandas as pd
import numpy as np
import random

In [2]:
x = mido.MidiFile(filename='bach_846.mid')
x

<midi file 'bach_846.mid' type 1, 11 tracks, 3164 messages>

In [3]:
p = x.play()

In [53]:
note = p.__next__()

In [105]:
mid.tracks[1][6].text

'bdca426d104a26ac9dcb070447587523'

In [101]:
mid = mido.MidiFile('bach_846.mid')

In [86]:
for x in mid.tracks[1][0:100]:
    print(x)

<meta message midi_port port=0 time=0>
<meta message track_name name='Piano right' time=0>
program_change channel=0 program=0 time=0
control_change channel=0 control=7 value=100 time=0
control_change channel=0 control=10 value=64 time=0
control_change channel=0 control=91 value=127 time=0
<meta message text text='bdca426d104a26ac9dcb070447587523' time=0>
note_on channel=0 note=67 velocity=56 time=241
note_on channel=0 note=67 velocity=0 time=120
note_on channel=0 note=72 velocity=60 time=0
note_on channel=0 note=72 velocity=0 time=120
note_on channel=0 note=76 velocity=63 time=0
note_on channel=0 note=76 velocity=0 time=108
note_on channel=0 note=67 velocity=50 time=12
note_on channel=0 note=67 velocity=0 time=120
note_on channel=0 note=72 velocity=47 time=0
note_on channel=0 note=72 velocity=0 time=120
note_on channel=0 note=76 velocity=47 time=0
note_on channel=0 note=76 velocity=0 time=120
note_on channel=0 note=67 velocity=56 time=240
note_on channel=0 note=67 velocity=0 time=120
n

In [85]:
mid.tracks[1]

<midi track 'Piano right' 869 messages>

NOTES:

Timing in MIDI files is centered around ticks and beats. A beat is the same as a quarter note. Beats are divided into ticks, the smallest unit of time in MIDI.


inside a track, it is delta time in ticks. This must be an integer.

Unlike music, tempo in MIDI is not given as beats per minute, but rather in microseconds per beat.
The default tempo is 500000 microseconds per beat, which is 120 beats per minute.

MIDI is a binary protocol. Each each message is encoded as a status byte followed by up to three data bytes



In [69]:
# http://www.piano-midi.de/midis/mozart/mz_311_1.mid
f = 'mz_311_1_format0.mid'

In [91]:
mid = mido.MidiFile(f)

480

In [110]:
upperBound = 102
lowerBound = 24

def midiToNoteMatrix(midifile):
    mid = mido.MidiFile(midifile)
    
    time_left = [track[0].time for track in mid.tracks]
    positions = [0 for track in mid.tracks]
    
    state_matrix = []
    span = upperBound - lowerBound
    time = 0
    
    state = [[0,0] for x in range(span)]
    state_matrix.append(state)
    
    while True:
        if time % (mid.ticks_per_beat / 4) == (mid.ticks_per_beat / 8):
            oldstate = state
            state = [[oldstate[x][0],0] for x in range(span)]
            state_matrix.append(state)
            
        for i in range(len(time_left)):
            while time_left[i] == 0:
                track = mid.tracks[i]
                pos = positions[i]
                        
                event = track[pos]
                #fix NoteEvent missing
                if event.type in ['note_on', 'note_off']:
                    if (event.note < lowerBound) or (event.note >= upperBound):
                        pass
                        # print "Note {} at time {} out of bounds (ignoring)".format(evt.pitch, time)
                    else:
                        if event.type == 'note_off' or event.velocity == 0:
                            state[event.note-lowerBound] = [0, 0]
                        else:
                            state[event.note-lowerBound] = [1, 1]
                elif event.type == 'time_signature':
                    if event.numerator not in (2, 4):
                        # We don't want to worry about non-4 time signatures. Bail early!
                        # print "Found time signature event {}. Bailing!".format(evt)
                        return state_matrix

                try:
                    time_left[i] = track[pos + 1].time
                    positions[i] += 1
                except IndexError:
                    time_left[i] = None

            if time_left[i] is not None:
                time_left[i] -= 1

        if all(t is None for t in time_left):
            break

        time += 1

    return state_matrix


In [112]:
sm = midiToNoteMatrix(f)

In [126]:
def noteStateMatrixToMidi(state_matrix, name="example"):
    state_matrix = np.asarray(state_matrix)
    mid = mido.MidiFile()
    track = mido.MidiTrack()
    mid.tracks.append(track)
    
    span = upperBound-lowerBound
    tickscale = 120  # beats per minute
    
    last_command_time = 0
    prev_state = [[0,0] for x in range(span)]
    for time, state in enumerate(state_matrix + [prev_state[:]]):  
        note_on_list = []
        note_off_list = []
        for i in range(span):
            note = state[i]
            prev_note = prev_state[i]
            # was there a preceding note?
            if prev_note[0] == 1:
                # the note is now off
                if note[0] == 0:
                    note_off_list.append(i)
                # new note 
                elif note[1] == 1:
                    note_off_list.append(i)
                    note_on_list.append(i)
            # no preceding note, but a note is now played
            elif note[0] == 1:
                note_on_list.append(i)
        for note in note_off_list:
            track.append(mido.Message('note_off', time=(time-last_command_time)*tickscale, note=note+lowerBound))
            last_command_time = time
        for note in note_on_list:
            track.append(mido.Message('note_on', time=(time-last_command_time)*tickscale, velocity=40, note=note+lowerBound))
            last_command_time = time
            
        prev_state = state
    
    eot = mido.MetaMessage('end_of_track', time=1)
    track.append(eot)

    mid.save(f'{name}.mid')


In [127]:
noteStateMatrixToMidi(sm)

In [1]:
def buildContext(state):
    """ Determine which notes are being played within a given state"""
    context = [0]*12
    for note, notestate in enumerate(state):
        if notestate[0] == 1:
            pitchclass = (note + lowerBound) % 12  # Which note to play
            context[pitchclass] += 1
    return context

In [2]:
def buildBeat(time):
    """Build the time signature"""
    return [2*x-1 for x in [time%2, (time//2)%2, (time//4)%2, (time//8)%2]]

In [3]:
def getListDefault(lst, i, default):
    """ Get a list value at index or default"""
    try:
        return lst[i]
    except IndexError:
        return default


def noteInputForm(note, state, context, beat):
    """ Convert a note to the input format"""
    # Determine which note is being "played"
    pitchclass = (note + lowerBound) % 12
    
    # Generate list with bits specifying the notes being played
    part_pitchclass = [int(i == pitchclass) for i in range(12)]
    
    # Concatenate the note states for the previous vicinity
    part_prev_vicinity = list(itertools.chain.from_iterable((getListDefault(state, note+i, [0,0]) for i in range(-12, 13))))

    part_context = context[pitchclass:] + context[:pitchclass]

    # Return the giant list for input
    return [note] + part_pitchclass + part_prev_vicinity + part_context + beat + [0]

def noteStateSingleToInputForm(state,time):
    beat = buildBeat(time)
    context = buildContext(state)
    return [noteInputForm(note, state, context, beat) for note in range(len(state))]

def noteStateMatrixToInputForm(statematrix):
    inputform = [noteStateSingleToInputForm(state,time) for time,state in enumerate(statematrix) ]
    return inputform

# MODEL STUFF HERE

In [6]:
batch_width = 10 # number of sequences in a batch
batch_len = 16*8 # length of each sequence
division_len = 16 # interval between possible start locations

def loadPieces(dirpath):

    pieces = {}

    for fname in os.listdir(dirpath):
        if fname[-4:] not in ('.mid','.MID'):
            continue

        name = fname[:-4]

        outMatrix = midiToNoteStateMatrix(os.path.join(dirpath, fname))
        if len(outMatrix) < batch_len:
            continue

        pieces[name] = outMatrix
        print(f"Loaded {name}")

    return pieces

def getPieceSegment(pieces):
    piece_output = random.choice(pieces.values())
    start = random.randrange(0,len(piece_output)-batch_len,division_len)
    # print "Range is {} {} {} -> {}".format(0,len(piece_output)-batch_len,division_len, start)

    seg_out = piece_output[start:start+batch_len]
    seg_in = noteStateMatrixToInputForm(seg_out)

    return seg_in, seg_out

def getPieceBatch(pieces):
    i,o = zip(*[getPieceSegment(pieces) for _ in range(batch_width)])
    return numpy.array(i), numpy.array(o)

def trainPiece(model,pieces,epochs,start=0):
    for i in range(start,start+epochs):
        error = model.update_fun(*getPieceBatch(pieces))
        if i % 100 == 0:
            print(f"epoch {i}, error={error}")
        if i % 500 == 0 or (i % 100 == 0 and i < 1000):
            xIpt, xOpt = map(numpy.array, getPieceSegment(pieces))
            noteStateMatrixToMidi(numpy.concatenate((numpy.expand_dims(xOpt[0], 0), model.predict_fun(batch_len, 1, xIpt[0])), axis=0),'output/sample{}'.format(i))
            pickle.dump(model.learned_config,open(f'output/params{i}.p', 'wb'))

# Recurrent Neural Network
- What is it?
- Why is it used?
- How does it work?
